## 1. Load packages and data

In [4]:
# import packages
import os 

import re
from datetime import datetime

import pytz

import pandas as pd
import numpy as np

import ast

import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import array_contains
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, udf, lit
from sparknlp.pretrained import PretrainedPipeline 


import matplotlib.pyplot as plt
import seaborn as sns

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import emojis
from translate import Translator

import sparknlp


In [5]:
import sparknlp 

spark = sparknlp.start(m1=True)

print("Spark NLP version")
sparknlp.version()
print("Apache Spark version")
spark.version

:: loading settings :: url = jar:file:/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/wouterdewitte/.ivy2/cache
The jars for the packages stored in: /Users/wouterdewitte/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp-m1_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b83eff4f-f59e-497a-9b05-8038cb97b6d7;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-m1_2.12;4.2.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found

22/11/30 21:56:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark NLP version
Apache Spark version


'3.3.1'

In [6]:
# import findspark
import findspark

# initialize findspark with spark directory

#ALWAYS HAVE TO BE CHANGED 
findspark.init("/Users/wouterdewitte/spark/")

# import pyspark
import pyspark
# create spark context
#sc = pyspark.SparkContext()
# create spark session 
#spark = pyspark.sql.SparkSession(sc)

In [4]:
#set this path to your path, for some reason I have an error 
#reading in all the files
#path_json = ".././../data/Topic_vegan/*.json"

# use this if you want all the tweet files, but this is usually too large
#df_json = spark.read.json(path_json)

In [7]:
list_brands = ["healthyfood",
               "healthylifestyle",
               "vegan",
               "keto",
               "ketodiet",
               "ketolifestyle",
               "veganism",
               "vegetarian"]
from re import search



data_dir = ".././../data/Topic_vegan/"
tweet_files = [os.path.join(data_dir, obs) for obs in os.listdir(data_dir)]



files_brand = [file for file in tweet_files if (file.find(list_brands[2]) != -1)]
files_brand               
               
df_json = spark.read.option("multiline","true").json(files_brand)  
df_json.count()

22/11/30 21:57:30 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


3428559

In [8]:
# select interesting features
import pyspark.sql.functions as F

df = df_json.select(F.col("user.name"),
                    F.col("user.screen_name"),
                    F.col("created_at"), 
                    F.col("full_text"))
df.limit(10).toPandas()

,name,screen_name,created_at,full_text
0,のり/Nori,nori_k_629,Mon Apr 04 10:09:55 +0000 2022,RT @ohmpawatt: เพื่อนๆคิดถึงผมมั้ยยย ถ้าคิดถึง...
1,alice,myn4meizalize,Mon Apr 04 10:09:54 +0000 2022,RT @mynameisnanon: คิดถึงกันป่าว ถ้าคิดถึงต้อง...
2,Karen Reed 🌸,kandk670,Mon Apr 04 10:09:54 +0000 2022,@trudiebakescake Organic coconut oil in a jar ...
3,ハル):),patlnwza55,Mon Apr 04 10:09:52 +0000 2022,RT @ohmpawatt: เพื่อนๆคิดถึงผมมั้ยยย ถ้าคิดถึง...
4,alice,myn4meizalize,Mon Apr 04 10:09:52 +0000 2022,RT @ohmpawatt: เพื่อนๆคิดถึงผมมั้ยยย ถ้าคิดถึง...
5,ﾌｧﾙﾄﾞﾗっ子organic有機💙💻,organic_yusai,Mon Apr 04 10:09:52 +0000 2022,マジでピンチ助けて自転車ガガガガ
6,のり/Nori,nori_k_629,Mon Apr 04 10:09:50 +0000 2022,RT @mynameisnanon: คิดถึงกันป่าว ถ้าคิดถึงต้อง...
7,｡◕‿◕｡𝑱𝒆 𝒕'𝒂𝒊𝒎𝒆 🐶🧡✨,MyFnlovely97,Mon Apr 04 10:09:50 +0000 2022,RT @ohmpawatt: เพื่อนๆคิดถึงผมมั้ยยย ถ้าคิดถึง...
8,Sang™,asan_gk,Mon Apr 04 10:09:50 +0000 2022,RT @NotechAna: Am I the only one who types in ...
9,Trysia ):)▪︎never let me go▪︎,Winnie_thephuu,Mon Apr 04 10:09:48 +0000 2022,RT @ohmpawatt: เพื่อนๆคิดถึงผมมั้ยยย ถ้าคิดถึง...


## 2. Preprocess Data

In [9]:
# https://developer.twitter.com/en/docs/twitter-ads-api/timezones
# function to convert Twitter date string format
def getDate(date):
    if date is not None:
        return str(datetime.strptime(date,'%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo=pytz.UTC).strftime("%Y-%m-%d %H:%M:%S"))
    else:
        return None

# UDF declaration
date_udf = F.udf(getDate, StringType())

# apply udf
df = df.withColumn('post_created_at', F.to_utc_timestamp(date_udf("created_at"), "UTC"))

In [10]:
#drop duplicates and retweets 
df = df.filter(~F.col("full_text").startswith("RT"))\
                        .drop_duplicates()
#sorting such when dropping later we only keep the most recent post 
df = df.sort("post_created_at", ascending=False)
#removing spam accounts 
df = df.drop_duplicates(["full_text", "screen_name"])

#df.printSchema()
#df.count() #1340938

In [11]:
# define function to count hashtags
def get_hashtags(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "#" in word:
            counter += 1
    return(counter) 

# define function to count mentions
def get_mentions(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "@" in word:
            counter += 1
    return(counter)

# define function to count exclamation marks
def get_exclamation_marks(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "!" in word:
            counter += 1
    return(counter)

# define function to count number of emojis used
import emojis
def emoji_counter(text):
    nr_emojis = emojis.count(text)
    return(nr_emojis)
# register functions as udf
get_hashtags_UDF = F.udf(get_hashtags, IntegerType())
get_mentions_UDF = F.udf(get_mentions, IntegerType())
get_exclamation_marks_UDF = F.udf(get_exclamation_marks, IntegerType())
emoji_counter_udf = F.udf(emoji_counter, IntegerType())


In [12]:
twitter_df = df.withColumn("emoji_count", emoji_counter_udf("full_text")) \
                            .withColumn("text_tokenized", F.split("full_text", " ")) \
                            .withColumn("num_words", F.size("text_tokenized")) \
                            .withColumn("num_hashtags", get_hashtags_UDF("text_tokenized")) \
                            .withColumn("num_mentions", get_mentions_UDF("text_tokenized")) \
                            .withColumn("num_exclamation_marks", get_exclamation_marks_UDF("text_tokenized"))
twitter_df.show()

+--------------------+---------------+--------------------+--------------------+-------------------+-----------+--------------------+---------+------------+------------+---------------------+
|                name|    screen_name|          created_at|           full_text|    post_created_at|emoji_count|      text_tokenized|num_words|num_hashtags|num_mentions|num_exclamation_marks|
+--------------------+---------------+--------------------+--------------------+-------------------+-----------+--------------------+---------+------------+------------+---------------------+
| Follow the Vegans Ⓥ|  vegan_v_vegan|Sat May 14 00:55:...|!\n#vegan #GoVega...|2022-05-14 00:55:33|          0|[!\n#vegan, #GoVe...|        4|           3|           0|                    1|
|🌱Veg-In-Out Mark...| veginoutmarket|Sat Jan 15 07:17:...|! We will be open...|2022-01-15 07:17:18|          0|[!, We, will, be,...|       35|          21|           0|                    3|
|         Mix 93.8 FM|       Mix938FM|Wed

In [13]:
# define function to clean text
def clean_text(string):
    
    # define numbers
    NUMBERS = '0123456789'
    PUNCT = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    
    # convert text to lower case
    cleaned_string = string.lower()
    
    # remove URLS
    cleaned_string = re.sub(r'http\S+', ' ', cleaned_string)
    
    # replace emojis by words
    cleaned_string = emojis.decode(cleaned_string)
    cleaned_string = cleaned_string.replace(":"," ").replace("_"," ")
    cleaned_string = ' '.join(cleaned_string.split())
    
    # remove numbers
    cleaned_string = "".join([char for char in cleaned_string if char not in NUMBERS])
    
    # remove punctuation
    cleaned_string = "".join([char for char in cleaned_string if char not in PUNCT])
    
    # remove words conisting of one character (or less)
    cleaned_string = ' '.join([w for w in cleaned_string.split() if len(w) > 1])

    # return
    return(cleaned_string) 
clean_text_udf = F.udf(clean_text, StringType())


In [14]:
twitter_df = df.withColumn("text", clean_text_udf(F.col("full_text")))

In [13]:
#from sparknlp.pretrained import PretrainedPipeline 
#pipeline = PretrainedPipeline("translate_mul_en", lang = "xx")

translate_mul_en download started this may take some time.
Approx size to download 279,5 MB
[ / ]translate_mul_en download started this may take some time.
Approximate size to download 279,5 MB
Download done! Loading the resource.
[ — ]

2022-11-30 19:18:19.823250: W external/org_tensorflow/tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


[OK!]


In [19]:
#twitter_df = pipeline.transform(twitter_df)

In [15]:
from googletrans import Translator

translator = Translator() 
translation = translator.translate("test", dest='en')

def translate(tweet):
    if len(tweet)!=0:
        translator = Translator() 
        translation = translator.translate(tweet, dest = 'en')
        return(translation.text)
    return None

translate_udf = F.udf(translate, StringType())

twitter_df = twitter_df.withColumn("translation", translate_udf("text"))

twitter_df.show()

22/11/30 22:00:55 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 29 (TID 4671): Attempting to kill Python Worker
+--------------------+---------------+--------------------+--------------------+-------------------+--------------------+--------------------+
|                name|    screen_name|          created_at|           full_text|    post_created_at|                text|         translation|
+--------------------+---------------+--------------------+--------------------+-------------------+--------------------+--------------------+
| Follow the Vegans Ⓥ|  vegan_v_vegan|Sat May 14 00:55:...|!\n#vegan #GoVega...|2022-05-14 00:55:33| vegan govegan dairy| vegan govegan dairy|
|🌱Veg-In-Out Mark...| veginoutmarket|Sat Jan 15 07:17:...|! We will be open...|2022-01-15 07:17:18|we will be open a...|we will be open a...|
|         Mix 93.8 FM|       Mix938FM|Wed Sep 07 09:11:...|!! Daily Updates ...|2022-09-07 09:11:40|daily updates tas...|daily updates tas...|
| 

In [21]:
twitter_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- screen_name: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- full_text: string (nullable = true)
 |-- post_created_at: timestamp (nullable = true)
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: i

In [22]:
#twitter_df = twitter_df.withColumn("translation", twitter_df.translation.result)

In [28]:
#twitter_df = twitter_df.drop("document","sentence")

In [16]:
twitter_df.show()

22/11/30 22:02:32 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 41 (TID 4965): Attempting to kill Python Worker
+--------------------+---------------+--------------------+--------------------+-------------------+--------------------+--------------------+
|                name|    screen_name|          created_at|           full_text|    post_created_at|                text|         translation|
+--------------------+---------------+--------------------+--------------------+-------------------+--------------------+--------------------+
| Follow the Vegans Ⓥ|  vegan_v_vegan|Sat May 14 00:55:...|!\n#vegan #GoVega...|2022-05-14 00:55:33| vegan govegan dairy| vegan govegan dairy|
|🌱Veg-In-Out Mark...| veginoutmarket|Sat Jan 15 07:17:...|! We will be open...|2022-01-15 07:17:18|we will be open a...|we will be open a...|
|         Mix 93.8 FM|       Mix938FM|Wed Sep 07 09:11:...|!! Daily Updates ...|2022-09-07 09:11:40|daily updates tas...|daily updates tas...|
| 

# 3. Topic Modelling

https://github.com/maobedkova/TopicModelling_PySpark_SparkNLP/blob/master/Topic_Modelling_with_PySpark_and_Spark_NLP.ipynb

https://www.johnsnowlabs.com/spark-nlp/


## 3.1 Spark NLP pipeline

### 3.1.1 Basic NLP pipeline

DocumentAssembler converts data into Spark NLP annotation format that can be used by Spark NLP annotators.

In [17]:
from sparknlp.base import DocumentAssembler
documentAssembler = DocumentAssembler() \
                .setInputCol("translation") \
                .setOutputCol('document')

Next, we tokenize the data with Tokenizer

In [18]:
from sparknlp.annotator import Tokenizer
tokenizer = Tokenizer() \
     .setInputCols(['document']) \
     .setOutputCol('tokenized')

We clean out the data and lower it with Normalizer

In [19]:
from sparknlp.annotator import Normalizer
normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized') \
     .setLowercase(True)

We are going to lemmatize our text with pretrained lemming model provided by Spark NLP. We can access this model with LemmatizerModel.

In [20]:
from sparknlp.annotator import LemmatizerModel
lemmatizer = LemmatizerModel.pretrained() \
     .setInputCols(['normalized']) \
     .setOutputCol('lemmatized')

lemma_antbnc download started this may take some time.
Approximate size to download 907,6 KB
[ / ]lemma_antbnc download started this may take some time.
Approximate size to download 907,6 KB
Download done! Loading the resource.
[OK!]


Spark NLP doesn't provide stop word list, hence, we will use nltk package to download stop words for English.

In [21]:
from nltk.corpus import stopwords
eng_stopwords = stopwords.words('english')

In [22]:
from sparknlp.annotator import StopWordsCleaner
stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemmatized']) \
     .setOutputCol('unigrams') \
     .setStopWords(eng_stopwords)

In addition to unigrams, it is good to use n-grams for topic modelling as well since they help to better refine topics. We can get n-grams with NGramGenerator in Spark NLP.

In [23]:
from sparknlp.annotator import NGramGenerator

ngrammer = NGramGenerator() \
    .setInputCols(['lemmatized']) \
    .setOutputCol('ngrams') \
    .setN(3) \
    .setEnableCumulative(True) \
    .setDelimiter('_')

We already have our basic NLP pipeline for topic modelling with all necessary steps. However, let's use POS tagger in order to improve our processed data for topic modelling even more with POS tagged data later. For this, we are going to use pretrained POS tagging model provided by Spark NLP. We can access the model with PerceptronModel.

In [24]:
from sparknlp.annotator import PerceptronModel
pos_tagger = PerceptronModel.pretrained('pos_anc') \
     .setInputCols(['document', 'lemmatized']) \
     .setOutputCol('pos')

pos_anc download started this may take some time.
Approximate size to download 3,9 MB
[ / ]pos_anc download started this may take some time.
Approximate size to download 3,9 MB
Download done! Loading the resource.
[OK!]


Now we have everything in Spark NLP annotation format. To be able to process the data further, we need to tranform data with Finisher.

In [25]:
from sparknlp.base import Finisher
finisher = Finisher() \
     .setInputCols(['unigrams', 'ngrams', 'pos'])

Now we are ready to input everything into a pipeline. Pipeline functionality is accessible with PySpark.

In [26]:
from pyspark.ml import Pipeline
pipeline = Pipeline() \
     .setStages([documentAssembler,
                 tokenizer,
                 normalizer,
                 lemmatizer,
                 stopwords_cleaner,
                 pos_tagger,
                 ngrammer,
                 finisher])

In [27]:
processed_review = pipeline.fit(twitter_df).transform(twitter_df)

In [28]:
processed_review.limit(5).show()

22/11/30 22:04:22 WARN PythonUDFRunner: Detected deadlock while completing task 6.0 in stage 59 (TID 5296): Attempting to kill Python Worker
22/11/30 22:04:22 WARN PythonUDFRunner: Detected deadlock while completing task 1.0 in stage 59 (TID 5291): Attempting to kill Python Worker
22/11/30 22:04:22 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 59 (TID 5290): Attempting to kill Python Worker


22/11/30 22:04:24 WARN PythonUDFRunner: Detected deadlock while completing task 4.0 in stage 59 (TID 5294): Attempting to kill Python Worker
22/11/30 22:04:24 WARN PythonUDFRunner: Detected deadlock while completing task 2.0 in stage 59 (TID 5292): Attempting to kill Python Worker
22/11/30 22:04:24 WARN PythonUDFRunner: Detected deadlock while completing task 3.0 in stage 59 (TID 5293): Attempting to kill Python Worker
22/11/30 22:04:24 WARN PythonUDFRunner: Detected deadlock while completing task 5.0 in stage 59 (TID 5295): Attempting to kill Python Worker
22/11/30 22:04:24 WARN PythonUDFRunner: Detected deadlock while completing task 7.0 in stage 59 (TID 5297): Attempting to kill Python Worker


22/11/30 22:04:54 WARN PythonUDFRunner: Detected deadlock while completing task 8.0 in stage 59 (TID 5298): Attempting to kill Python Worker
+--------------------+--------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                name|   screen_name|          created_at|           full_text|    post_created_at|                text|         translation|   finished_unigrams|     finished_ngrams|        finished_pos|
+--------------------+--------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| Follow the Vegans Ⓥ| vegan_v_vegan|Sat May 14 00:55:...|!\n#vegan #GoVega...|2022-05-14 00:55:33| vegan govegan dairy| vegan govegan dairy|[vegan, govegan, ...|[vegan, govegan, ...|        [NN, NN, NN]|
|🌱Veg-In-Out Mark...|veginoutmarket|Sat

### 3.1.2 Extended NLP pipeline

Up to now, we have our data in a form of unigrams that are lemmatized, with no stop words in there. I think it is a good idea to incorporate n-grams into our NLP pipeline. We obtained n-grams as one step of our pipeline but now n-grams are messy and have a lot of questionable combinations in there. To tackle this problem, let's filter out strange combinations of words in n-grams based on their POS tags. We can imagine a list of viable combinations like ADJ + NOUN so let's restrict our POS combinations in n-grams to this list. Plus, we can also exclude some POS tags from our unigrams to ensure that we don't use functional words for topic modelling (they can be partially covered by stop words but probably not fully).

Doing this POS-based filtering will significantly reduce the vocabulary size for topic modelling which will speed up the whole processing.

Let's start this processing. First, we need to join all our POS tags obtained previously.

In [29]:
from pyspark.sql import types as T

udf_join_arr = F.udf(lambda x: ' '.join(x), T.StringType())
processed_review  = processed_review.withColumn('finished_pos', udf_join_arr(F.col('finished_pos')))

Then we start another Spark NLP pipeline in order to get POS tag n-grams that correspond to word n-grams. We start with convertation into Spark NLP annotation format.

In [30]:
pos_documentAssembler = DocumentAssembler() \
     .setInputCol('finished_pos') \
     .setOutputCol('pos_document')

Then, we tokenize our POS tags.

In [31]:
pos_tokenizer = Tokenizer() \
     .setInputCols(['pos_document']) \
     .setOutputCol('pos')

And generate n-grams from them in the same way we did that for words.

In [32]:
pos_ngrammer = NGramGenerator() \
    .setInputCols(['pos']) \
    .setOutputCol('pos_ngrams') \
    .setN(3) \
    .setEnableCumulative(True) \
    .setDelimiter('_')

Lastly, we are ready to get POS tags ngrams with Finisher.

In [33]:
pos_finisher = Finisher() \
     .setInputCols(['pos', 'pos_ngrams'])

We create this new Spark NLP pipeline...

In [34]:
pos_pipeline = Pipeline() \
     .setStages([pos_documentAssembler,                  
                 pos_tokenizer,
                 pos_ngrammer,  
                 pos_finisher])

... and again fit it and transform the data.

In [35]:
processed_review = pos_pipeline.fit(processed_review).transform(processed_review)

Let's look what kind of data we have to operate with.

In [42]:
processed_review.columns

['name',
 'screen_name',
 'created_at',
 'full_text',
 'post_created_at',
 'text',
 'translation',
 'finished_unigrams',
 'finished_ngrams',
 'finished_pos',
 'finished_pos_ngrams']

And these are our word n-grams with their corresponding pos n-grams.

In [42]:
processed_review.select('finished_ngrams', 'finished_pos_ngrams').limit(5).show()

22/11/30 22:16:07 WARN PythonUDFRunner: Detected deadlock while completing task 3.0 in stage 89 (TID 5898): Attempting to kill Python Worker
22/11/30 22:16:07 WARN PythonUDFRunner: Detected deadlock while completing task 7.0 in stage 89 (TID 5902): Attempting to kill Python Worker


Traceback (most recent call last):
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 187, in manager
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 730, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 593, in read_int
    length = stream.read(4)
ConnectionResetError: [Errno 54] Connection reset by peer


22/11/30 22:16:09 WARN PythonUDFRunner: Detected deadlock while completing task 7.0 in stage 89 (TID 5902): Attempting to kill Python Worker
22/11/30 22:16:09 WARN PythonUDFRunner: Detected deadlock while completing task 6.0 in stage 89 (TID 5901): Attempting to kill Python Worker
22/11/30 22:16:09 WARN PythonUDFRunner: Detected deadlock while completing task 4.0 in stage 89 (TID 5899): Attempting to kill Python Worker


Traceback (most recent call last):
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 187, in manager
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 730, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 595, in read_int
    raise EOFError
EOFError


22/11/30 22:16:13 WARN PythonUDFRunner: Detected deadlock while completing task 1.0 in stage 89 (TID 5896): Attempting to kill Python Worker
22/11/30 22:16:13 WARN PythonUDFRunner: Detected deadlock while completing task 5.0 in stage 89 (TID 5900): Attempting to kill Python Worker


Traceback (most recent call last):
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 187, in manager
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 730, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 593, in read_int
    length = stream.read(4)
ConnectionResetError: [Errno 54] Connection reset by peer
Traceback (most recent call last):
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 187, in manager
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/

22/11/30 22:16:17 WARN PythonUDFRunner: Detected deadlock while completing task 2.0 in stage 89 (TID 5897): Attempting to kill Python Worker


22/11/30 22:16:47 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 89 (TID 5895): Attempting to kill Python Worker


22/11/30 22:20:01 WARN PythonUDFRunner: Detected deadlock while completing task 8.0 in stage 89 (TID 5903): Attempting to kill Python Worker
+--------------------+--------------------+
|     finished_ngrams| finished_pos_ngrams|
+--------------------+--------------------+
|[this, afternoon,...|[DT, NN, VBG, JJ,...|
|[sweet, potato, a...|[JJ, NN, CC, NN, ...|
|[creamy, mushroom...|[NN, NN, NN, NN, ...|
|[polypieter, vegg...|[NN, NN, VB, RB, ...|
|[enrich, with, na...|[NN, IN, JJ, NN, ...|
+--------------------+--------------------+



22/12/01 05:19:09 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 993145 ms exceeds timeout 120000 ms
22/12/01 05:19:09 WARN SparkContext: Killing executors is not supported by current scheduler.


Now we are ready to filter out not useful for topic modelling analysis POS tags from our data. Let's create the function that does it for unigrams first. We create the custom Python function and then transform it to PySpark UDF to be used on Spark dataframe.

In [36]:
def filter_pos(words, pos_tags):
    return [word for word, pos in zip(words, pos_tags) 
            if pos in ['JJ', 'NN', 'NNS', 'VB', 'VBP']]

udf_filter_pos = F.udf(filter_pos, T.ArrayType(T.StringType()))

Then, we apply this function on columns with unigrams and their POS tags to get filtered unigrams in a separate dataframe column.

In [37]:
processed_review = processed_review.withColumn('filtered_unigrams',
                                               udf_filter_pos(F.col('finished_unigrams'), 
                                                              F.col('finished_pos')))

That is how our filtered unigrams look like.

In [46]:
processed_review.select('filtered_unigrams').limit(5).show(truncate=90)

22/11/30 21:49:25 WARN PythonUDFRunner: Detected deadlock while completing task 5.0 in stage 81 (TID 5607): Attempting to kill Python Worker


22/11/30 21:49:27 WARN PythonUDFRunner: Detected deadlock while completing task 2.0 in stage 81 (TID 5604): Attempting to kill Python Worker
22/11/30 21:49:27 WARN PythonUDFRunner: Detected deadlock while completing task 3.0 in stage 81 (TID 5605): Attempting to kill Python Worker
22/11/30 21:49:27 WARN PythonUDFRunner: Detected deadlock while completing task 7.0 in stage 81 (TID 5609): Attempting to kill Python Worker


Traceback (most recent call last):
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 187, in manager
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 730, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 595, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):                                  (3 + 6) / 9]
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 187, in manager
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/pyth

22/11/30 21:49:29 WARN PythonUDFRunner: Detected deadlock while completing task 6.0 in stage 81 (TID 5608): Attempting to kill Python Worker


Traceback (most recent call last):
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 187, in manager
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 730, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 595, in read_int
    raise EOFError
EOFError


22/11/30 21:50:11 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 81 (TID 5602): Attempting to kill Python Worker
22/11/30 21:50:11 WARN PythonUDFRunner: Detected deadlock while completing task 1.0 in stage 81 (TID 5603): Attempting to kill Python Worker


22/11/30 21:50:13 WARN PythonUDFRunner: Detected deadlock while completing task 4.0 in stage 81 (TID 5606): Attempting to kill Python Worker


22/11/30 21:53:35 WARN PythonUDFRunner: Detected deadlock while completing task 8.0 in stage 81 (TID 5610): Attempting to kill Python Worker
+------------------------------------------------------------------------------------------+
|                                                                         filtered_unigrams|
+------------------------------------------------------------------------------------------+
|[livestreaming, conversation, asefrid, book, govegan, livelikeagorilla, veganfortheanim...|
|                                   [sweet, potato, fritter, mint, dip, vegan, plant, base]|
|                           [creamy, mushroom, bucatini, vegan, veganforlife, forevervegan]|
|[polypieter, veggie, already, vegan, almost, impossible, get, hey, dont, ingredient, ve...|
|                             [enrich, origin, ingredient, certify, vegan, wink, fragrance]|
+------------------------------------------------------------------------------------------+



It is time to filter out improper POS combinations of n-grams. We create the custom function in the same manner as before. Since we deal with bi- and trigrams, we need to restrict tags for both.

In [38]:
def filter_pos_combs(words, pos_tags):
    return [word for word, pos in zip(words, pos_tags) 
            if (len(pos.split('_')) == 2 and \
                pos.split('_')[0] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                 pos.split('_')[1] in ['JJ', 'NN', 'NNS']) \
            or (len(pos.split('_')) == 3 and \
                pos.split('_')[0] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                 pos.split('_')[1] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                  pos.split('_')[2] in ['NN', 'NNS'])]
    
udf_filter_pos_combs = F.udf(filter_pos_combs, T.ArrayType(T.StringType()))

And we call the function on word and POS n-grams.

In [39]:
processed_review = processed_review.withColumn('filtered_ngrams',
                                               udf_filter_pos_combs(F.col('finished_ngrams'),
                                                                    F.col('finished_pos_ngrams')))

Below is what we get after filtering for n-grams.

In [49]:
processed_review.select('filtered_ngrams').limit(5).show(truncate=90)

22/11/30 21:55:08 ERROR PythonUDFRunner: Python worker exited unexpectedly (crashed)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 666, in main
    eval_type = read_int(infile)
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 595, in read_int
    raise EOFError
EOFError

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:559)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:86)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:68)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:512)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/var/folders/x2/hymzscwj7pg_t42hpfwjm20m0000gn/T/ipykernel_93262/2024449930.py", line 9, in translate
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/googletrans/client.py", line 210, in translate
    data, response = self._translate(text, dest, src, kwargs)
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/googletrans/client.py", line 108, in _translate
    r = self.client.get(url, params=params)
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/httpx/_client.py", line 755, in get
    return self.request(
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/httpx/_client.py", line 600, in request
    return self.send(
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/httpx/_client.py", line 628, in send
    response.close()
  File "/Users/w

Now we have unigrams and n-grams stored in different columns in the dataframe. Let's combine them together.

In [40]:
from pyspark.sql.functions import concat

processed_review = processed_review.withColumn('final', 
                                               concat(F.col('filtered_unigrams'), 
                                                      F.col('filtered_ngrams')))

And this is our final look of the data.

In [43]:
processed_review.select('final').limit(5).show(truncate=90)

22/12/01 07:39:06 WARN PythonUDFRunner: Detected deadlock while completing task 7.0 in stage 106 (TID 6205): Attempting to kill Python Worker


Traceback (most recent call last):
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 187, in manager
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 730, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 593, in read_int
    length = stream.read(4)
ConnectionResetError: [Errno 54] Connection reset by peer


22/12/01 07:39:08 WARN PythonUDFRunner: Detected deadlock while completing task 5.0 in stage 106 (TID 6203): Attempting to kill Python Worker


22/12/01 07:39:10 WARN PythonUDFRunner: Detected deadlock while completing task 6.0 in stage 106 (TID 6204): Attempting to kill Python Worker


22/12/01 07:39:18 WARN PythonUDFRunner: Detected deadlock while completing task 2.0 in stage 106 (TID 6200): Attempting to kill Python Worker
22/12/01 07:39:18 WARN PythonUDFRunner: Detected deadlock while completing task 3.0 in stage 106 (TID 6201): Attempting to kill Python Worker


...

22/12/01 07:39:22 WARN PythonUDFRunner: Detected deadlock while completing task 4.0 in stage 106 (TID 6202): Attempting to kill Python Worker


Traceback (most recent call last):
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 187, in manager
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 730, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/wouterdewitte/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 595, in read_int
    raise EOFError
EOFError


22/12/01 07:40:10 WARN PythonUDFRunner: Detected deadlock while completing task 1.0 in stage 106 (TID 6199): Attempting to kill Python Worker


22/12/01 07:40:14 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 106 (TID 6198): Attempting to kill Python Worker


22/12/01 07:42:50 WARN PythonUDFRunner: Detected deadlock while completing task 8.0 in stage 106 (TID 6206): Attempting to kill Python Worker
+------------------------------------------------------------------------------------------+
|                                                                                     final|
+------------------------------------------------------------------------------------------+
|[livestreaming, conversation, asefrid, book, govegan, livelikeagorilla, veganfortheanim...|
|[sweet, potato, fritter, mint, dip, vegan, plant, base, sweet_potato, courgette_fritter...|
|[creamy, mushroom, bucatini, vegan, veganforlife, forevervegan, creamy_mushroom, mushro...|
|[polypieter, veggie, already, vegan, almost, impossible, get, hey, dont, ingredient, ve...|
|[enrich, origin, ingredient, certify, vegan, wink, fragrance, natural_origin, origin_in...|
+------------------------------------------------------------------------------------------+



## 3.2 Vectorization

Now we are set to vectorization of our data. First, we will proceed with TF (term frequency) vectorization with CountVectorizer in PySpark. We fit tf dictionary and then transform the data to vectors of counts.

In [44]:
from pyspark.ml.feature import CountVectorizer

tfizer = CountVectorizer(inputCol='final', outputCol='tf_features')
tf_model = tfizer.fit(processed_review)
tf_result = tf_model.transform(processed_review)

After we get TF results, we can account for words that are frequent for all the documents. We can use IDF (inverse document frequency) to lower score of such words.

In [ ]:
from pyspark.ml.feature import IDF

idfizer = IDF(inputCol='tf_features', outputCol='tf_idf_features')
idf_model = idfizer.fit(tf_result)
tfidf_result = idf_model.transform(tf_result)

22/11/30 12:32:19 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB


22/11/30 12:44:02 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB


## LDA

Finally, we are ready to model topics in our data with LDA (Latent Dirichlet Allocation). To use the algorithm, we have to provide the number of topics we presume our data contains and the number of iterations for the LDA algorithm. Then, we initialize the model and train it.

In [ ]:
tfidf_result.show()

22/11/30 12:46:12 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB


+--------------------+---------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                name|    screen_name|          created_at|           full_text|    post_created_at|        cleaned_text|   finished_unigrams|     finished_ngrams|        finished_pos| finished_pos_ngrams|   filtered_unigrams|     filtered_ngrams|               final|         tf_features|     tf_idf_features|
+--------------------+---------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| Follow the Vegans Ⓥ|  vegan_v_vegan|Sat May 14 00:55:...|!\n#vega

In [ ]:
from pyspark.ml.clustering import LDA

num_topics = 6
max_iter = 10

lda = LDA(k=num_topics, maxIter=max_iter, featuresCol='tf_idf_features').setTopicDistributionCol("topicDistributionCol")
lda_model = lda.fit(tfidf_result)
transformed = lda_model.transform(tfidf_result)

22/11/30 12:49:24 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB


22/11/30 12:58:43 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
22/11/30 12:58:43 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
22/11/30 12:58:44 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
22/11/30 12:58:44 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/11/30 12:58:44 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


22/11/30 12:58:53 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB


22/11/30 12:58:54 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
22/11/30 12:58:55 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB


22/11/30 12:58:58 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB


...

22/11/30 12:59:17 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB


22/11/30 12:59:17 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
22/11/30 12:59:18 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB


22/11/30 12:59:19 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB


22/11/30 12:59:19 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
22/11/30 12:59:20 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB


22/11/30 12:59:21 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB


In [ ]:
transformed.show()

22/11/30 13:01:04 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB


+--------------------+---------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                name|    screen_name|          created_at|           full_text|    post_created_at|        cleaned_text|   finished_unigrams|     finished_ngrams|        finished_pos| finished_pos_ngrams|   filtered_unigrams|     filtered_ngrams|               final|         tf_features|     tf_idf_features|topicDistributionCol|
+--------------------+---------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| Fo

In [ ]:
def get_topic(input_list):
    max = input_list[0]
    index = 0
    for i in range(1,len(input_list)):
        if input_list[i] > max:
            max = input_list[i]
            index = i
    return index

get_topic_udf = udf(lambda z: get_topic(z), IntegerType())

In [ ]:
transformed = transformed.withColumn('topic', get_topic_udf("topicDistributionCol"))

In [ ]:
freq_month = transformed.withColumn("year", year(df["post_created_at"]))
freq_month = freq_month.withColumn("month", month(df["post_created_at"]))

freq_month = freq_month.groupBy('year', 'month', 'topic').agg(countDistinct("full_text"))\
               .withColumnRenamed("count(full_text)", "freq") \
                    .sort('year', 'month', ascending = True)
freq_month = freq_month.select(concat_ws('_',freq_month.year, freq_month.month)\
                            .alias('date'), 'topic', 'freq').toPandas()


22/11/30 13:03:10 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB


22/11/30 13:20:41 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB


22/11/30 13:20:42 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB
22/11/30 13:20:42 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB


22/11/30 13:20:43 WARN DAGScheduler: Broadcasting large task binary with size 20.5 MiB


In [ ]:
freq_month

,date,topic,freq
0,2021_10,5,10339
1,2021_10,4,8625
2,2021_10,2,10528
3,2021_10,1,16279
4,2021_10,3,8261
...,...,...,...
73,2022_10,3,684
74,2022_10,2,651
75,2022_10,0,596
76,2022_10,5,681


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
                   x=freq_month["date"],
                   y=freq_month["topic"],
                   z=freq_month["freq"],
                   hoverongaps = True))
fig.show()

In [ ]:
freq_month

,date,topic,freq
0,2021_10,5,10339
1,2021_10,4,8625
2,2021_10,2,10528
3,2021_10,1,16279
4,2021_10,3,8261
...,...,...,...
73,2022_10,3,684
74,2022_10,2,651
75,2022_10,0,596
76,2022_10,5,681


In [ ]:
print((transformed.count(), len(transformed.columns)))

(1340938, 17)


To be able to see words that characterize the defined topics, we need to convert word ids into actual words with the custom function. This function will again be converted to PySpark UDF to be used on our topic dataframe.

In [ ]:
vocab = tf_model.vocabulary

def get_words(token_list):
     return [vocab[token_id] for token_id in token_list]
       
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))

Let's define the number of top words per topic we would like to see and extract the words with our function.

In [ ]:
num_top_words = 10

topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=90)

+-----+------------------------------------------------------------------------------------------+
|topic|                                                                                topicWords|
+-----+------------------------------------------------------------------------------------------+
|    0| [vegan, organic, joy, joy_joy, amp, food, healthylifestyle, healthyfood, health, healthy]|
|    1|                       [vegan, organic, vegano, vegetarian, rofl, keto, go, eat, im, make]|
|    2|                            [que, vegano, vegan, el, heart, tea, keto, organic, spice, en]|
|    3|           [vegan, keto, und, ich, diet, die, check_mark, healthylifestyle, health, check]|
|    4|[small_blue, blue_diamond, small_blue_diamond, small, blue, diamond, organic, diamond_s...|
|    5|                           [vegan, organic, sob, amp, good, make, soap, sob_sob, diet, im]|
+-----+------------------------------------------------------------------------------------------+



In [ ]:
topics_rdd = topics.rdd
topics_words = topics_rdd\
       .map(lambda row: row['termIndices'])\
       .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
       .collect()
for idx, topic in enumerate(topics_words):
    print("topic: {}".format(idx))
    print("*"*25)
    for word in topic:
       print(word)
    print("*"*25)

topic: 0
*************************
vegan
organic
joy
joy_joy
amp
food
healthylifestyle
healthyfood
health
healthy
*************************
topic: 1
*************************
vegan
organic
vegano
vegetarian
rofl
keto
go
eat
im
make
*************************
topic: 2
*************************
que
vegano
vegan
el
heart
tea
keto
organic
spice
en
*************************
topic: 3
*************************
vegan
keto
und
ich
diet
die
check_mark
healthylifestyle
health
check
*************************
topic: 4
*************************
small_blue
blue_diamond
small_blue_diamond
small
blue
diamond
organic
diamond_small
vegan
food
*************************
topic: 5
*************************
vegan
organic
sob
amp
good
make
soap
sob_sob
diet
im
*************************


22/11/30 14:04:30 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 139945 ms exceeds timeout 120000 ms
22/11/30 14:04:30 WARN SparkContext: Killing executors is not supported by current scheduler.


## 3.3 Visualization